<a href="https://colab.research.google.com/github/adicangit/Financial-Investment-Research-CoPilot/blob/main/Ai_Agents_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install All the needed packages

In [ ]:
pip install dotenv llama-index-tools-tavily-research llama-index llama-hub tavily-python llama_index.llms.groq llama-index-tools-yahoo-finance pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━

# Import all the needed packages

In [ ]:
from llama_index.llms.groq import Groq
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage, MessageRole

# Different types of agents
from llama_index.core.agent.workflow import (
    ReActAgent, # Run
    FunctionAgent, # Run
    AgentWorkflow #MultiAgent
)

from llama_index.core.tools import(
    BaseTool,
    FunctionTool,
    QueryEngineTool
)
from llama_index.core import PromptTemplate
from llama_index.core.agent import ReActChatFormatter
from llama_index.tools.yahoo_finance import YahooFinanceToolSpec
from IPython.display import Markdown
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream
)

from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    draw_all_possible_flows
)

import pyvis

from llama_index.tools.tavily_research.base import TavilyToolSpec
from pydantic import BaseModel, Field
from typing import Union

from llama_index.core.agent import FunctionCallingAgent # Run

from llama_index.core.memory import (
    ChatMemoryBuffer,
)
from llama_index.embeddings.openai import OpenAIEmbedding

import yfinance as yf
import datetime as dt

from llama_index.core.tools import FunctionTool
from datetime import datetime
from llama_index.core.tools import ToolMetadata, FunctionTool
from llama_index.core.workflow import WorkflowCheckpointer
from llama_index.core.workflow import Context

import os
from dotenv import load_dotenv
load_dotenv()
import json

# Groq LLM

In [ ]:
llm = Groq(model="llama-3.3-70b-versatile", api_key = "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")

### Code to check if the llm is working

In [ ]:
messages = [
    ChatMessage(
        role="system", content="You are a highly knowledgeable financial analyst specializing in the stock market."
    ),
    ChatMessage(role="user", content="what is todays date"),
]
resp = llm.chat(messages)

In [ ]:
display(Markdown(resp.message.blocks[0].text))

Today's date is April 12, 2025.

# Tools
- Some tools are being used from APIs while some tools were manually defined
- (To do) LLMs(LLama models from groq) can’t handle large texts at once especially when calling basic_stock_info for more than one stock, so we need a more optimized basic stock info.

In [ ]:
finance_tools = YahooFinanceToolSpec().to_tool_list()
finance_tools = finance_tools[:-1]

In [ ]:
tavily_tool = TavilyToolSpec(api_key = "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").to_tool_list()

In [ ]:
search = tavily_tool[0]

In [ ]:
print(finance_tools[0].metadata.description) # total 6 functions

balance_sheet(ticker: str) -> str

        Return the balance sheet of the stock.

        Args:
          ticker (str): the stock ticker to be given to yfinance

        


In [ ]:
search.metadata

ToolMetadata(description='search(query: str, max_results: Optional[int] = 6) -> List[llama_index.core.schema.Document]\n\n        Run query through Tavily Search and return metadata.\n\n        Args:\n            query: The query to search for.\n            max_results: The maximum number of results to return.\n\n        Returns:\n            results: A list of dictionaries containing the results:\n                url: The url of the result.\n                content: The content of the result.\n\n        ', name='search', fn_schema=<class 'llama_index.core.tools.utils.search'>, return_direct=False)

In [ ]:
def returns(ticker: str, start_date: str, end_date:str) -> float:
    """
    Return the 'Returns' of a stock between start_date and end_date.


    Args:
      ticker (str): the stock ticker to be given to yfinance
      start_date (str): the starting date from which the returns have to be calculated in the format YYYY-MM-DD
      end_data (str): the ending date till which the returns have to be calculated YYYY-MM-DD

    """
    stock = yf.Ticker(ticker)
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    data = stock.history(start=start_date, end=end_date, auto_adjust=True)

    if len(data) == 1:
        return round(((data['Open'].iloc[0]-data['Close'].iloc[0])/data['Close'].iloc[0])*100,2)

    return round(( (data['Close'].iloc[-1]-data['Close'].iloc[0])/data['Close'].iloc[0])*100,2)

In [ ]:
def price_history(ticker: str, start_date: str, end_date:str):
    """
    Return the list of a stocks' price between start_date and end_date.

    Args:
      ticker (str): the stock ticker to be given to yfinance
      start_date (str): the starting date from which the returns have to be calculated in the format YYYY-MM-DD
      end_data (str): the ending date till which the returns have to be calculated YYYY-MM-DD

    """
    stock = yf.Ticker(ticker)
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    data = stock.history(start=start_date, end=end_date, auto_adjust=True)
    return data.to_json()
    # data = data['Close'].tolist()

    # return data

In [ ]:
price_history_tool = FunctionTool.from_defaults(
    price_history,
)

In [ ]:
# agent = ReActAgent(
#     tools = [today_date_tool,price_history_tool],
#     llm=llm,
#     verbose=True,
#     system_prompt="You're a helpful assistant.",
# )

In [ ]:
# handler = agent.run("1 week price history of nvda")

# async for event in handler.stream_events():
#     if isinstance(event, AgentStream):
#         print(event.delta, end="", flush=True)

In [ ]:
from datetime import datetime
import yfinance as yf

def CAGR(ticker: str, start_date: str, end_date: str, period: int) -> float:
    """
    Return the CAGR of a stock between start_date and end_date.

    Args:
      ticker (str): Stock ticker symbol.
      start_date (str): Start date in "YYYY-MM-DD" format.
      end_date (str): End date in "YYYY-MM-DD" format.
      period (int): Number of years over which to calculate CAGR.

    Returns:
      float: CAGR in percentage.
    """
    stock = yf.Ticker(ticker)
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    data = stock.history(start=start_date, end=end_date, auto_adjust=True)

    if data.empty:
        raise ValueError(f"No stock data available for {ticker} in the given date range.")

    initial_price = data['Close'].iloc[0]
    final_price = data['Close'].iloc[-1]

    period = int(period)

    cagr = ((final_price / initial_price) ** (1 / period)) - 1
    return cagr * 100

In [ ]:
returns_tool = FunctionTool.from_defaults(
    returns
)

In [ ]:
CAGR_tool = FunctionTool.from_defaults(
    CAGR
)

In [ ]:
def get_date() -> str:
        """
        A function to return todays date.
        Call this before any other functions if you are unaware of the current date

        Returns:
        str: Todays date in YYYY-MM-DD
        """
        return datetime.today().strftime('%Y-%m-%d')


today_date_tool = FunctionTool.from_defaults(
    fn=get_date,
)

# Agent Team


## Team Members

In [ ]:
summary_agent = FunctionAgent(
    name="SummaryAgent",
    description="Handles general conversations and provides basic stock summaries. Delegates specific tasks to specialized agents.",
    system_prompt=(
        "You are the SummaryAgent, responsible for answering general questions and providing a basic stock summary. "
        "For stock performance metrics like returns and CAGR, you must delegate to the PerformanceAgent. "
        "For stock-related news and ONLY when asked for news, you must delegate to the NewsAgent. "
        "Once the specialized agent completes its task, take control back and summarize the response for the user."
    ),
    llm=llm,
    tools=[finance_tools[3],today_date_tool],
    can_handoff_to=["PerformanceAgent", "NewsAgent","CAGRAgent"]
)


In [ ]:
CAGR_agent = FunctionAgent(
    name="CAGRAgent",
    description="Handles stock CAGR calculations. Returns control to SummaryAgent once done.",
    system_prompt=(
        "You are the CAGRAgent, an expert in stock CAGR calculations. "
        "You can calculate stock CAGR. "
        "Once you finish the necessary calculations, present the information in a structured way."
    ),
    llm=llm,
    tools=[CAGR_tool, today_date_tool],
    can_handoff_to=["SummaryAgent"]
)


In [ ]:
performance_agent = ReActAgent(
    name="PerformanceAgent",
    description="Handles stock returns calculations, Returns control to SummaryAgent once done.",
    system_prompt=(
        "You are the PerformanceAgent, an expert in stock returns calculations. "
        "You can calculate stock returns for different time periods (e.g., 1-day, 1-month, 1-year, 10-year) "
        "You can also provide the historical prices of a given stock"
        "Once you provide the necessary calculations, return control to the SummaryAgent so it can present the information in a structured way."
    ),
    llm=llm,
    tools=[returns_tool, today_date_tool,price_history_tool],
    can_handoff_to=["SummaryAgent"]
)


In [ ]:
news_agent = FunctionAgent(
    name="NewsAgent",
    description="Finds recent news related to a given stock or company. Returns control to SummaryAgent after retrieving news.",
    system_prompt=(
        "You are the NewsAgent, an expert at finding recent news about stocks, companies, and financial markets. "
        "When asked about stock-related news, search for relevant articles and summarize key insights. "
        "After retrieving the news, return control to the SummaryAgent to provide a final response to the user."
    ),
    llm=llm,
    tools=[search],
    can_handoff_to=["SummaryAgent"]
)


## Team Workflow

In [ ]:
agent_workflow = AgentWorkflow(
    agents=[summary_agent, news_agent, performance_agent, CAGR_agent],
    root_agent=summary_agent.name,
    initial_state={
        "todays date": f'{get_date()}'
    },
)

## User-Agent Chat

In [ ]:
ctx = Context(agent_workflow)
w_cptr = WorkflowCheckpointer(workflow=agent_workflow)

while True:
  input_user_msg = input("User: ")
  print('\n')
  if input_user_msg == "exit":
    break

  handler = w_cptr.run(user_msg=input_user_msg, ctx=ctx)
  await handler
  result = handler.result()
  print('Agent: ',result.response.blocks[0].text)
  print('\n')

User: provide me an overview on microsofts stock


Agent:  Microsoft Corporation is a technology company that develops and supports software, services, devices, and solutions worldwide. The company's stock has a current price of $388.45, with a 52-week low of $344.79 and a 52-week high of $468.35. Microsoft has a market capitalization of $2.88 trillion and a dividend yield of 0.85%. The company's revenue has grown by 12.3% in the last year, and its earnings have grown by 10.2%. Microsoft's profit margins are 35.43%, and its operating margins are 45.58%. The company has a strong balance sheet, with a quick ratio of 1.20 and a current ratio of 1.35. Overall, Microsoft is a well-established company with a strong track record of growth and profitability.


User: exit




## To Debug for a Specific Prompt

In [ ]:
ctx = Context(agent_workflow)

in_text = input("Input: ")

handler = agent_workflow.run(
    user_msg=f"{in_text}",
    ctx=ctx
)

current_agent = None
current_tool_calls = ""

async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
            break
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")
        # 10 year cagr for nvda

Input: whats the news on microsoft stock

🤖 Agent: SummaryAgent

🛠️  Planning to use tools: ['handoff']
🔨 Calling Tool: handoff
  With arguments: {'reason': 'user requested news on microsoft stock', 'to_agent': 'NewsAgent'}
🔧 Tool Result (handoff):
  Arguments: {'reason': 'user requested news on microsoft stock', 'to_agent': 'NewsAgent'}
  Output: Agent NewsAgent is now handling the request due to the following reason: user requested news on microsoft stock.
Please continue with the current request.

🤖 Agent: NewsAgent

🛠️  Planning to use tools: ['search']
🔨 Calling Tool: search
  With arguments: {'query': 'Microsoft stock news', 'max_results': 3}
🔧 Tool Result (search):
  Arguments: {'query': 'Microsoft stock news', 'max_results': 3}
  Output: [Document(id_='e59d24ab-9d6b-484e-b00d-62cd72183cb0', embedding=None, metadata={'url': 'https://www.marketwatch.com/investing/stock/msft'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{ke